In [784]:
import blgndr
import pandas as pd

In [785]:
tabel_alternatif = blgndr.clean()

In [786]:
tabel_alternatif

,ADMINISTRASI NIAGA - UI (IPS),SISTEM INFORMASI - UI (IPA),HUKUM - UI (IPS),ILMU SEJARAH - UI (IPS),FISIKA - UI (IPA),ILMU KOMUNIKASI - UI (IPS),FARMASI - UI (IPA),TEKNIK KIMIA - UI (IPA),KEDOKTERAN GIGI - UI (IPA),MATEMATIKA - UI (IPA),...,ANTROPOLOGI SOSIAL - UNAIR (IPS),PEND. APOTEKER - UNAIR (IPA),PEND. DOKTER - UNAIR (IPA),KIMIA - UNAIR (IPA),FAKULTAS KEDOKTERAN GIGI - UNAIR (IPA),FISIKA - UNAIR (IPA),ILMU KESEHATAN MASYARAKAT - UNAIR (IPA),STATISTIKA - UNAIR (IPA),KEDOKTERAN GIGI - UNAIR (IPA),PEND. NERS - UNAIR (IPA)
Geografi,84.839303,87.000000,86.709826,84.946434,84.727782,84.666667,84.839303,84.839303,84.839303,84.839303,...,83.919652,84.839303,84.839303,84.839303,84.839303,84.839303,84.839303,84.839303,84.839303,84.839303
Biologi,84.769668,89.000000,84.769668,84.769668,84.476967,84.769668,84.769668,86.500000,85.000000,85.905359,...,84.769668,91.000000,98.000000,89.000000,88.000000,73.000000,92.000000,84.769668,84.769668,84.769668
Fisika,84.698593,88.000000,84.698593,82.465729,87.369859,84.698593,84.698593,91.000000,84.800000,87.284507,...,84.698593,91.000000,98.000000,91.000000,88.000000,73.000000,90.000000,84.698593,84.698593,84.698593
Kimia,85.152057,88.000000,85.152057,85.152057,83.915206,85.152057,85.152057,93.500000,85.800000,87.088620,...,85.152057,94.000000,98.000000,90.000000,88.000000,73.000000,87.000000,85.152057,85.152057,85.152057
Sosiologi,85.293746,85.293746,89.323436,85.431249,85.183798,88.431249,85.293746,85.293746,85.293746,85.940862,...,84.146873,85.293746,85.293746,85.293746,85.293746,85.293746,85.194267,85.293746,85.293746,85.293746
Inggris,84.697708,83.000000,85.424427,83.899236,84.969771,86.232569,84.697708,86.000000,85.400000,86.592131,...,85.348854,90.000000,93.000000,91.000000,91.000000,73.000000,83.000000,84.697708,84.697708,84.697708
Ekonomi,84.867923,84.867923,88.966981,85.955974,84.890861,86.666667,84.867923,90.433962,84.897180,84.691181,...,83.933962,84.867923,84.867923,88.000000,84.867923,76.014208,82.000000,84.867923,84.867923,84.867923
Matematika,84.236315,91.000000,82.559079,82.745438,86.123632,83.666667,84.236315,90.000000,88.400000,91.769231,...,78.906746,92.000000,93.000000,87.000000,90.000000,65.300000,85.000000,84.236315,84.236315,84.236315
Indonesia,85.393215,85.000000,87.098304,86.464405,84.439321,87.000000,85.393215,92.000000,86.200000,86.261017,...,86.196607,90.000000,93.000000,87.000000,97.000000,73.000000,88.000000,85.393215,85.393215,85.393215


In [787]:
decision_matrix = blgndr.getDecisionMatrix(tabel_alternatif)

In [788]:
decision_matrix

,Geografi,Biologi,Fisika,Kimia,Sosiologi,Inggris,Ekonomi,Matematika,Indonesia
ADMINISTRASI NIAGA - UI (IPS),"[0.8483930315858026, 0.1516069684141974]","[0.8476966770093084, 0.1523033229906916]","[0.8469859299691537, 0.15301407003084633]","[0.8515205714226723, 0.14847942857732765]","[0.8529374595544092, 0.14706254044559075]","[0.8469770826702545, 0.1530229173297455]","[0.8486792322169479, 0.15132076778305215]","[0.8423631545305261, 0.15763684546947387]","[0.8539321488941178, 0.14606785110588216]"
SISTEM INFORMASI - UI (IPA),"[0.87, 0.13]","[0.89, 0.10999999999999999]","[0.88, 0.12]","[0.88, 0.12]","[0.8529374595544092, 0.14706254044559075]","[0.83, 0.17000000000000004]","[0.8486792322169479, 0.15132076778305215]","[0.91, 0.08999999999999997]","[0.85, 0.15000000000000002]"
HUKUM - UI (IPS),"[0.8670982578964507, 0.13290174210354933]","[0.8476966770093084, 0.1523033229906916]","[0.8469859299691537, 0.15301407003084633]","[0.8515205714226723, 0.14847942857732765]","[0.8932343648886024, 0.10676563511139758]","[0.8542442706675637, 0.1457557293324363]","[0.889669808054237, 0.11033019194576299]","[0.8255907886326316, 0.17440921136736842]","[0.8709830372235294, 0.12901696277647057]"
ILMU SEJARAH - UI (IPS),"[0.8494643438619341, 0.15053565613806585]","[0.8476966770093084, 0.1523033229906916]","[0.8246572866461024, 0.17534271335389762]","[0.8515205714226723, 0.14847942857732765]","[0.8543124865181365, 0.14568751348186348]","[0.8389923608900848, 0.16100763910991522]","[0.8595597440723161, 0.14044025592768394]","[0.8274543848435088, 0.17254561515649125]","[0.8646440496313725, 0.13535595036862746]"
FISIKA - UI (IPA),"[0.8472778248127646, 0.1527221751872354]","[0.8447696677009309, 0.15523033229906913]","[0.8736985929969154, 0.12630140700308456]","[0.8391520571422673, 0.16084794285773274]","[0.85183798032515, 0.14816201967485]","[0.8496977082670254, 0.1503022917329746]","[0.8489086129273776, 0.15109138707262237]","[0.8612363154530526, 0.13876368454694743]","[0.8443932148894118, 0.15560678511058823]"
...,...,...,...,...,...,...,...,...,...
FISIKA - UNAIR (IPA),"[0.8483930315858026, 0.1516069684141974]","[0.73, 0.27]","[0.73, 0.27]","[0.73, 0.27]","[0.8529374595544092, 0.14706254044559075]","[0.73, 0.27]","[0.7601420765027322, 0.23985792349726776]","[0.653, 0.347]","[0.73, 0.27]"
ILMU KESEHATAN MASYARAKAT - UNAIR (IPA),"[0.8483930315858026, 0.1516069684141974]","[0.92, 0.07999999999999996]","[0.9, 0.09999999999999998]","[0.87, 0.13]","[0.8519426672618162, 0.14805733273818378]","[0.83, 0.17000000000000004]","[0.82, 0.18000000000000005]","[0.85, 0.15000000000000002]","[0.88, 0.12]"
STATISTIKA - UNAIR (IPA),"[0.8483930315858026, 0.1516069684141974]","[0.8476966770093084, 0.1523033229906916]","[0.8469859299691537, 0.15301407003084633]","[0.8515205714226723, 0.14847942857732765]","[0.8529374595544092, 0.14706254044559075]","[0.8469770826702545, 0.1530229173297455]","[0.8486792322169479, 0.15132076778305215]","[0.8423631545305261, 0.15763684546947387]","[0.8539321488941178, 0.14606785110588216]"
KEDOKTERAN GIGI - UNAIR (IPA),"[0.8483930315858026, 0.1516069684141974]","[0.8476966770093084, 0.1523033229906916]","[0.8469859299691537, 0.15301407003084633]","[0.8515205714226723, 0.14847942857732765]","[0.8529374595544092, 0.14706254044559075]","[0.8469770826702545, 0.1530229173297455]","[0.8486792322169479, 0.15132076778305215]","[0.8423631545305261, 0.15763684546947387]","[0.8539321488941178, 0.14606785110588216]"


In [789]:
entropy_matrix = blgndr.getEntropyMatrix(decision_matrix)

In [790]:
entropy_matrix

,Geografi,Biologi,Fisika,Kimia,Sosiologi,Inggris,Ekonomi,Matematika,Indonesia
ADMINISTRASI NIAGA - UI (IPS),0.303214,0.304607,0.306028,0.296959,0.294125,0.306046,0.302642,0.315274,0.292136
SISTEM INFORMASI - UI (IPA),0.260000,0.220000,0.240000,0.240000,0.294125,0.340000,0.302642,0.180000,0.300000
HUKUM - UI (IPS),0.265803,0.304607,0.306028,0.296959,0.213531,0.291511,0.220660,0.348818,0.258034
ILMU SEJARAH - UI (IPS),0.301071,0.304607,0.350685,0.296959,0.291375,0.322015,0.280881,0.345091,0.270712
FISIKA - UI (IPA),0.305444,0.310461,0.252603,0.321696,0.296324,0.300605,0.302183,0.277527,0.311214
...,...,...,...,...,...,...,...,...,...
FISIKA - UNAIR (IPA),0.303214,0.540000,0.540000,0.540000,0.294125,0.540000,0.479716,0.694000,0.540000
ILMU KESEHATAN MASYARAKAT - UNAIR (IPA),0.303214,0.160000,0.200000,0.260000,0.296115,0.340000,0.360000,0.300000,0.240000
STATISTIKA - UNAIR (IPA),0.303214,0.304607,0.306028,0.296959,0.294125,0.306046,0.302642,0.315274,0.292136
KEDOKTERAN GIGI - UNAIR (IPA),0.303214,0.304607,0.306028,0.296959,0.294125,0.306046,0.302642,0.315274,0.292136


In [791]:
knowledge_matrix = blgndr.getKnowledgeMatrix(entropy_matrix)

In [792]:
knowledge_matrix

,Geografi,Biologi,Fisika,Kimia,Sosiologi,Inggris,Ekonomi,Matematika,Indonesia
ADMINISTRASI NIAGA - UI (IPS),0.848393,0.847697,0.846986,0.851521,0.852937,0.846977,0.848679,0.842363,0.853932
SISTEM INFORMASI - UI (IPA),0.870000,0.890000,0.880000,0.880000,0.852937,0.830000,0.848679,0.910000,0.850000
HUKUM - UI (IPS),0.867098,0.847697,0.846986,0.851521,0.893234,0.854244,0.889670,0.825591,0.870983
ILMU SEJARAH - UI (IPS),0.849464,0.847697,0.824657,0.851521,0.854312,0.838992,0.859560,0.827454,0.864644
FISIKA - UI (IPA),0.847278,0.844770,0.873699,0.839152,0.851838,0.849698,0.848909,0.861236,0.844393
...,...,...,...,...,...,...,...,...,...
FISIKA - UNAIR (IPA),0.848393,0.730000,0.730000,0.730000,0.852937,0.730000,0.760142,0.653000,0.730000
ILMU KESEHATAN MASYARAKAT - UNAIR (IPA),0.848393,0.920000,0.900000,0.870000,0.851943,0.830000,0.820000,0.850000,0.880000
STATISTIKA - UNAIR (IPA),0.848393,0.847697,0.846986,0.851521,0.852937,0.846977,0.848679,0.842363,0.853932
KEDOKTERAN GIGI - UNAIR (IPA),0.848393,0.847697,0.846986,0.851521,0.852937,0.846977,0.848679,0.842363,0.853932


In [793]:
weight = blgndr.getWeight(knowledge_matrix)

In [794]:
weight

{'Geografi': 0.11,
 'Biologi': 0.11,
 'Fisika': 0.11,
 'Kimia': 0.11,
 'Sosiologi': 0.11,
 'Inggris': 0.11,
 'Ekonomi': 0.11,
 'Matematika': 0.11,
 'Indonesia': 0.11}

In [795]:
data_test = pd.read_csv("data/train-test/Splitted Test.csv", index_col = 0)

In [796]:
data_test

,Nama Panggilan,Jurusan Sekolah,Pilihan I,Pilihan II,Pilihan III,Jurusan Diterima,Geografi,Biologi,Fisika,Kimia,Sosiologi,Inggris,Ekonomi,Matematika,Indonesia
362,Rena,IPA,MANAJEMEN - UNAIR (IPS),HUKUM - UNAIR (IPS),MANAJEMEN - UNEJ (IPS),MANAJEMEN - UNAIR (IPS),84.839303,91.839691,91.897895,92.068108,85.293746,89.774143,84.867923,91.866061,90.953894
345,Alip,IPS,ILMU HUKUM - UNAIR (IPS),ILMU HUKUM - UNS (IPS),KOSONG,ILMU HUKUM - UNAIR (IPS),87.000000,84.769668,84.698593,85.152057,88.000000,86.000000,91.000000,82.000000,89.000000
71,Mimi,IPA,MATEMATIKA - UI (IPA),PENDIDIKAN MATEMATIKA - UNJ (IPA),KOSONG,MATEMATIKA - UI (IPA),84.839303,83.000000,86.000000,86.000000,85.293746,86.000000,84.867923,92.000000,84.000000
84,Nida,IPA,GEOGRAFI - UI (IPA),MANAJEMEN HUTAN - UI (IPS),SILVIKULTUR - IPB (IPA),GEOGRAFI - UI (IPA),84.839303,84.769668,84.698593,85.152057,85.293746,84.697708,84.867923,83.000000,85.393215
251,Syfa,IPA,KEDOKTERAN GIGI - UNPAD (IPA),PSIKOLOGI - UNPAD (IPS),KOSONG,KEDOKTERAN GIGI - UNPAD (IPA),84.839303,90.000000,80.000000,85.152057,85.293746,84.000000,82.515840,79.000000,85.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,Icha,IPA,BIOLOGI - UNPAD (IPA),KOSONG,BIOLOGI - UIN BANDUNG (IPA),BIOLOGI - UNPAD (IPA),84.839303,86.000000,82.000000,82.000000,85.293746,86.000000,87.613144,83.000000,80.000000
164,Evalinesca,IPA,ILMU PETERNAKAN - UNPAD (IPA),PERIKANAN - UNPAD (IPA),ILMU KELUARGA & KONSUMEN - ITB (IPS),ILMU PETERNAKAN - UNPAD (IPA),84.839303,84.769668,84.698593,85.152057,85.293746,84.697708,84.867923,84.236315,85.393215
275,Rifa,IPA,KIMIA - UNPAD (IPA),KOSONG,KOSONG,KIMIA - UNPAD (IPA),82.813400,90.000000,81.000000,92.000000,85.293746,79.000000,84.867923,88.000000,89.000000
20,Agustien,IPA,KEDOKTERAN - UI (IPA),GIZI - UI (IPA),KOSONG,KEDOKTERAN - UI (IPA),84.839303,91.000000,92.000000,93.000000,92.000000,93.000000,84.867923,86.000000,91.000000


In [797]:
deviasi = lambda tabel_alternatif, kriteria, alternatif, alternatif_compare: tabel_alternatif.loc[kriteria, alternatif] - tabel_alternatif.loc[kriteria, alternatif_compare]

In [798]:
def getNilaiSiswa(index, data_test):
    #assign nilai dari datatest
    nilai = {
        "Matematika": data_test.loc[index, "Matematika"],
        "Indonesia": data_test.loc[index, "Indonesia"],
        "Inggris": data_test.loc[index, "Inggris"],
    }
    for column in data_test.columns:
        if "ipa" in data_test.loc[index, "Jurusan Sekolah"].casefold():
            if column.casefold() == "fisika"\
            or column.casefold() == "kimia"\
            or column.casefold() == "biologi":
                nilai[column] = data_test.loc[index, column]
            elif column.casefold() == "ekonomi"\
            or column.casefold() == "geografi"\
            or column.casefold() == "sosiologi":
                nilai[column] = 0
        else:
            if column.casefold() == "ekonomi"\
            or column.casefold() == "geografi"\
            or column.casefold() == "sosiologi":
                nilai[column] = data_test.loc[index, column]
            elif column.casefold() == "fisika"\
            or column.casefold() == "kimia"\
            or column.casefold() == "biologi":
                nilai[column] = 0
                
    return nilai

In [799]:
def getPilihanSiswa(data_test, tabel_alternatif, index):
    pil_df = dict()
    for column in data_test:
        if column == "Pilihan I" or column == "Pilihan II" or column == "Pilihan III":
            if column not in pil_df:
                if "kosong" not in data_test.loc[index, column].casefold():
                    try:
                        pil_df[data_test.loc[index, column]] = tabel_alternatif[data_test.loc[index, column]].copy()
                    except:
                        pil_df[data_test.loc[index, column]] = [78] * 9

    pilihan_siswa = pd.DataFrame(pil_df, index = [kriteria for kriteria in tabel_alternatif.index])
    
    return pilihan_siswa

In [800]:
def getIndifferenceParams(nilai, pilihan_siswa):
    #menghitung nilai indifference sebagai parameter
    indifference = dict()
    for key in nilai.keys():
        min_val = abs(nilai[key] - pilihan_siswa.min(axis=1)[key])
        max_val = abs(nilai[key] - pilihan_siswa.max(axis=1)[key])
        if min_val > max_val:
            indifference[key] = max_val
        else:
            indifference[key] = min_val
            
    return indifference

In [801]:
def getPreferenceParams(nilai, pilihan_siswa):
    #menghitung nilai preference sebagai parameter
    preference = dict()
    for key in nilai.keys():
        min_val = abs(nilai[key] - pilihan_siswa.min(axis=1)[key])
        max_val = abs(nilai[key] - pilihan_siswa.max(axis=1)[key])
        if min_val > max_val:
            preference[key] = min_val
        else:
            preference[key] = max_val
            
    return preference

In [802]:
def getFungsiPreferensi(indifference, preference):
    global pilihan_siswa
    #mendapatkan fungsi preferensi
    fp_dataframe = dict()
    for alternatif in pilihan_siswa:
        for alternatif_compare in pilihan_siswa:
            fp_val_list = list()
            if alternatif == alternatif_compare:
                continue
            else:
                for kriteria in pilihan_siswa.index:
                    #mendapatkan nilai deviasi
                    dev = deviasi(pilihan_siswa, kriteria, alternatif, alternatif_compare)

                    #mendapatkan nilai fungsi preferensi
                    if dev <= indifference[kriteria]:
                        fp_value = 0
                    elif indifference[kriteria] < dev and dev <= preference[kriteria]:
                        fp_value = 1/2
                    elif dev > preference[kriteria]:
                        fp_value = 1

                    if "ips" in alternatif.casefold():
                        if kriteria.casefold() == "fisika"\
                        or kriteria.casefold() == "kimia"\
                        or kriteria.casefold() == "biologi":
                            fp_value = 0
                    elif "ipa" in alternatif.casefold():
                        if kriteria.casefold() == "ekonomi"\
                        or kriteria.casefold() == "geografi"\
                        or kriteria.casefold() == "sosiologi":
                            fp_value = 0
                    
#                     if fp_value != 0:
#                         print(fp_value)
                    fp_val_list.append(fp_value)
                else:
                    fp_dataframe[f"{alternatif} / {alternatif_compare}"] = fp_val_list
    else:
        fungsi_preferensi = pd.DataFrame(fp_dataframe, index = [kriteria for kriteria in pilihan_siswa.index])
        return fungsi_preferensi

In [803]:
def getIndexPrefGlob(fp_dataframe, weight):
    #menghitung index preferensi global
    index_prefglob_df = dict()
    for key in fp_dataframe.keys():
        index_prefglob_df[key] = [sum(fp_dataframe[key]) * weight["Matematika"]]
    else:
        index_prefglob = pd.DataFrame(index_prefglob_df, index = ["Index Preferensi Global"])
        return index_prefglob

In [804]:
def getEnteringFlow(index_prefglob):
    global pilihan_siswa
    temp_alternatif = ""
    skip = 0
    entering_flow_df = dict()
    #mendapatkan entering flow
    for alternatif in index_prefglob.columns:
        if skip == 0:
            temp_alternatif = alternatif.split("/")[1].strip()
            skip += 1

        if temp_alternatif == alternatif.split("/")[1].strip():
            if alternatif.split("/")[1].strip() not in entering_flow_df:
                entering_flow_df[alternatif.split("/")[1].strip()] = index_prefglob[alternatif]["Index Preferensi Global"] * (1/len(pilihan_siswa.columns))
            else:
                entering_flow_df[alternatif.split("/")[1].strip()] += index_prefglob[alternatif]["Index Preferensi Global"] * (1/len(pilihan_siswa.columns))
        else:
            entering_flow_df[alternatif.split("/")[1].strip()] = index_prefglob[alternatif]["Index Preferensi Global"] * (1/len(pilihan_siswa.columns))
            temp_alternatif = alternatif.split("/")[1].strip()
    else:
        entering_flow = pd.DataFrame(entering_flow_df, index=["Entering Flow"])
        return entering_flow

In [805]:
def getLeavingFlow(index_prefglob):
    global pilihan_siswa
    temp_alternatif = ""
    skip = 0
    leaving_flow_df = dict()
    #mendapatkan leaving flow
    for alternatif in index_prefglob.columns:
        if skip == 0:
            temp_alternatif = alternatif.split("/")[0].strip()
            skip += 1

        if temp_alternatif == alternatif.split("/")[0].strip():
            if alternatif.split("/")[0].strip() not in leaving_flow_df:
                leaving_flow_df[alternatif.split("/")[0].strip()] = index_prefglob[alternatif]["Index Preferensi Global"] * (1/len(pilihan_siswa.columns))
            else:
                leaving_flow_df[alternatif.split("/")[0].strip()] += index_prefglob[alternatif]["Index Preferensi Global"] * (1/len(pilihan_siswa.columns))
        else:
            leaving_flow_df[alternatif.split("/")[0].strip()] = index_prefglob[alternatif]["Index Preferensi Global"] * (1/len(pilihan_siswa.columns))
            temp_alternatif = alternatif.split("/")[0].strip()
    else:
        leaving_flow = pd.DataFrame(leaving_flow_df, index=["Leaving Flow"])
        return leaving_flow

In [806]:
def getNetFlow(leaving_flow, entering_flow, jurusan_sekolah):
    #menghitung net flow
    net_flow_df = dict()
    for alternatif in leaving_flow.columns:
        net_flow_df[alternatif] = leaving_flow.loc["Leaving Flow", alternatif] - entering_flow.loc["Entering Flow", alternatif]
    else:
        net_flow = pd.DataFrame(net_flow_df, index = ["Net Flow"])
        return net_flow.sort_values(by = "Net Flow", axis = 1, ascending = False)

In [807]:
hasil = dict()
stop = 1
for index in data_test.index:
    if stop > 100:
        break
    nilai = getNilaiSiswa(index, data_test)
    pilihan_siswa = getPilihanSiswa(data_test, tabel_alternatif, index)
    indifference = getIndifferenceParams(nilai, pilihan_siswa)
    preference = getPreferenceParams(nilai, pilihan_siswa)
    fungsi_preferensi = getFungsiPreferensi(indifference, preference)
    index_prefglob = getIndexPrefGlob(fungsi_preferensi, weight)
    entering_flow = getEnteringFlow(index_prefglob)
    leaving_flow = getLeavingFlow(index_prefglob)
    net_flow = getNetFlow(leaving_flow, entering_flow, data_test.loc[index, "Jurusan Diterima"])
    print(index)
    print(f"""
    Ekonomi : {data_test.loc[index, 'Ekonomi']}
    Sosiologi : {data_test.loc[index, 'Sosiologi']}
    Geografi : {data_test.loc[index, 'Geografi']}
    Matematika : {data_test.loc[index, 'Matematika']}
    Indonesia : {data_test.loc[index, 'Indonesia']}
    Inggris : {data_test.loc[index, 'Inggris']}
    Fisika : {data_test.loc[index, 'Fisika']}
    Kimia : {data_test.loc[index, 'Kimia']}
    Biologi : {data_test.loc[index, 'Biologi']}
          
    Pilihan Siswa
    {pilihan_siswa}
    
    Rekomendasi Sistem
    {net_flow}
    """)
    print()
    
    cek = list()
    if len(net_flow.columns) > 0:
        for rekomen in net_flow.columns:
            cek.append(rekomen)
    else: 
        cek.append((pilihan_siswa.columns)[0])
        
#     for i in range(3):
    if len(cek) > 0 :
        if f"Rekomendasi" not in hasil:
            hasil[f"Rekomendasi"] = [cek[0]]
        else:
            hasil[f"Rekomendasi"].append(cek[0])
        
        if "Riil" not in hasil:
            hasil["Riil"] = [data_test.loc[index, "Jurusan Diterima"]]
        else:
            hasil["Riil"].append(data_test.loc[index, "Jurusan Diterima"])
    stop += 1

362

    Ekonomi : 84.86792322169478
    Sosiologi : 85.29374595544094
    Geografi : 84.83930315858026
    Matematika : 91.8660606060606
    Indonesia : 90.95389408099688
    Inggris : 89.77414330218069
    Fisika : 91.8978947368421
    Kimia : 92.0681081081081
    Biologi : 91.83969072164948
          
    Pilihan Siswa
                MANAJEMEN - UNAIR (IPS)  HUKUM - UNAIR (IPS)  \
Geografi                  84.169652            84.839303   
Biologi                   86.537173            83.846445   
Fisika                    86.498418            84.132395   
Kimia                     86.881070            84.434705   
Sosiologi                 86.646873            85.293746   
Inggris                   87.367963            86.131806   
Ekonomi                   86.183962            84.867923   
Matematika                88.775594            83.824210   
Indonesia                 85.836777            85.595477   

            MANAJEMEN - UNEJ (IPS)  
Geografi                        78

    

58

    Ekonomi : 84.86792322169478
    Sosiologi : 85.29374595544094
    Geografi : 84.83930315858026
    Matematika : 84.23631545305261
    Indonesia : 85.39321488941178
    Inggris : 84.69770826702545
    Fisika : 84.69859299691538
    Kimia : 85.15205714226724
    Biologi : 84.76966770093084
          
    Pilihan Siswa
                ILMU ADM. FISKAL - UI (IPS)  ILMU ADM. PERPAJAKAN - UB (IPA)
Geografi                      84.839303                               78
Biologi                       84.769668                               78
Fisika                        84.698593                               78
Kimia                         85.152057                               78
Sosiologi                     85.293746                               78
Inggris                       84.697708                               78
Ekonomi                       84.867923                               78
Matematika                    84.236315                               78
Indones

316

    Ekonomi : 84.86792322169478
    Sosiologi : 85.29374595544094
    Geografi : 84.83930315858026
    Matematika : 84.23631545305261
    Indonesia : 85.39321488941178
    Inggris : 84.69770826702545
    Fisika : 84.69859299691538
    Kimia : 85.15205714226724
    Biologi : 84.76966770093084
          
    Pilihan Siswa
                MANAJEMEN - UNAIR (IPS)  PSIKOLOGI - UNAIR (IPS)  \
Geografi                  84.169652                85.806829   
Biologi                   86.537173                84.981361   
Fisika                    86.498418                83.125765   
Kimia                     86.881070                84.118334   
Sosiologi                 86.646873                86.005111   
Inggris                   87.367963                84.862595   
Ekonomi                   86.183962                85.279588   
Matematika                88.775594                84.652871   
Indonesia                 85.836777                85.269643   

            MANAJEMEN - UNES

369

    Ekonomi : 84.86792322169478
    Sosiologi : 84.0
    Geografi : 86.70135135135133
    Matematika : 90.0
    Indonesia : 88.0
    Inggris : 87.0
    Fisika : 84.0
    Kimia : 88.0
    Biologi : 89.0
          
    Pilihan Siswa
                PENDIDIKAN DOKTER - UNAIR (IPA)  PENDIDIKAN APOTEKER - UNAIR (IPA)
Geografi                          84.793319                          85.459986
Biologi                           88.589889                          86.179778
Fisika                            87.566198                          84.465729
Kimia                             86.384019                          86.101371
Sosiologi                         85.293746                          84.862497
Inggris                           87.565903                          85.465139
Ekonomi                           86.911949                          84.867923
Matematika                        89.078772                          86.157544
Indonesia                         85.797738      

248

    Ekonomi : 87.0
    Sosiologi : 86.0
    Geografi : 91.0
    Matematika : 83.0
    Indonesia : 86.0
    Inggris : 89.0
    Fisika : 84.69859299691538
    Kimia : 85.15205714226724
    Biologi : 84.76966770093084
          
    Pilihan Siswa
                ILMU HUKUM - UNPAD (IPS)  ILMU HUKUM - UNSOED (IPS)
Geografi                   86.279768                         78
Biologi                    84.769668                         78
Fisika                     84.698593                         78
Kimia                      85.152057                         78
Sosiologi                  86.097915                         78
Inggris                    87.232569                         78
Ekonomi                    86.622641                         78
Matematika                 83.245438                         78
Indonesia                  86.131072                         78
    
    Rekomendasi Sistem
              ILMU HUKUM - UNPAD (IPS)  ILMU HUKUM - UNSOED (IPS)
Net Flow     

159

    Ekonomi : 82.96522715381933
    Sosiologi : 85.29374595544094
    Geografi : 84.83930315858026
    Matematika : 79.0
    Indonesia : 83.0
    Inggris : 80.0
    Fisika : 81.0
    Kimia : 81.0
    Biologi : 83.0
          
    Pilihan Siswa
                STATISTIKA - UNPAD (IPA)
Geografi                   84.839303
Biologi                    84.179778
Fisika                     83.465729
Kimia                      83.768038
Sosiologi                  85.293746
Inggris                    83.131806
Ekonomi                    84.233691
Matematika                 83.745438
Indonesia                  84.595477
    
    Rekomendasi Sistem
    Empty DataFrame
Columns: []
Index: [Net Flow]
    

14

    Ekonomi : 84.86792322169478
    Sosiologi : 85.29374595544094
    Geografi : 84.83930315858026
    Matematika : 90.0
    Indonesia : 87.0
    Inggris : 93.0
    Fisika : 87.0
    Kimia : 92.0
    Biologi : 96.0
          
    Pilihan Siswa
                KEDOKTERAN - UI (IPA)
Geograf

6

    Ekonomi : 84.86792322169478
    Sosiologi : 85.29374595544094
    Geografi : 84.83930315858026
    Matematika : 84.23631545305261
    Indonesia : 85.39321488941178
    Inggris : 84.69770826702545
    Fisika : 84.69859299691538
    Kimia : 85.15205714226724
    Biologi : 84.76966770093084
          
    Pilihan Siswa
                FARMASI - UI (IPA)  TEKNIK PANGAN - IPB (IPA)
Geografi             84.839303                         78
Biologi              84.769668                         78
Fisika               84.698593                         78
Kimia                85.152057                         78
Sosiologi            85.293746                         78
Inggris              84.697708                         78
Ekonomi              84.867923                         78
Matematika           84.236315                         78
Indonesia            85.393215                         78
    
    Rekomendasi Sistem
              FARMASI - UI (IPA)  TEKNIK PANGAN - IPB (IPA)
Net

In [808]:
hasil_tabel = pd.DataFrame(hasil, index = data_test.index[:stop-1])

In [809]:
hasil_tabel

,Rekomendasi,Riil
362,MANAJEMEN - UNAIR (IPS),MANAJEMEN - UNAIR (IPS)
345,ILMU HUKUM - UNAIR (IPS),ILMU HUKUM - UNAIR (IPS)
71,MATEMATIKA - UI (IPA),MATEMATIKA - UI (IPA)
84,GEOGRAFI - UI (IPA),GEOGRAFI - UI (IPA)
251,KEDOKTERAN GIGI - UNPAD (IPA),KEDOKTERAN GIGI - UNPAD (IPA)
...,...,...
149,ILMU KOMUNIKASI - UI (IPS),ILMU KOMUNIKASI - UI (IPS)
11,KEDOKTERAN GIGI - UI (IPA),KEDOKTERAN GIGI - UI (IPA)
167,TEKNOLOGI INDUSTRI PERTANIAN - UNPAD (IPA),TEKNOLOGI INDUSTRI PERTANIAN - UNPAD (IPA)
310,BUDIDAYA PERAIRAN - UNAIR (IPA),BUDIDAYA PERAIRAN - UNAIR (IPA)


In [810]:
i = 1
for index in hasil_tabel.index:
    for column in hasil_tabel.columns:
        if column != "Riil":
            if hasil_tabel.loc[index, column] == hasil_tabel.loc[index, "Riil"]:
                pil1 = data_test.loc[index, 'Pilihan I']
                pil2 = data_test.loc[index, 'Pilihan II']
                pil3 = data_test.loc[index, 'Pilihan III']
                terima = hasil_tabel.loc[index, "Riil"]
                print(f"""{i}
                index: {index}
                rekomendasi sistem: {hasil_tabel.loc[index, column]}
                pilihan I: {pil1}
                pilihan II: {pil2}
                pilihan III: {pil3}
                diterima dijurusan: {terima}
                """)
                print()
                i += 1

1
                index: 362
                rekomendasi sistem: MANAJEMEN - UNAIR (IPS)
                pilihan I: MANAJEMEN - UNAIR (IPS)
                pilihan II: HUKUM - UNAIR (IPS)
                pilihan III: MANAJEMEN - UNEJ (IPS)
                diterima dijurusan: MANAJEMEN - UNAIR (IPS)
                

2
                index: 345
                rekomendasi sistem: ILMU HUKUM - UNAIR (IPS)
                pilihan I: ILMU HUKUM - UNAIR (IPS)
                pilihan II: ILMU HUKUM - UNS (IPS)
                pilihan III: KOSONG
                diterima dijurusan: ILMU HUKUM - UNAIR (IPS)
                

3
                index: 71
                rekomendasi sistem: MATEMATIKA - UI (IPA)
                pilihan I: MATEMATIKA - UI (IPA)
                pilihan II: PENDIDIKAN MATEMATIKA - UNJ (IPA)
                pilihan III: KOSONG
                diterima dijurusan: MATEMATIKA - UI (IPA)
                

4
                index: 84
                rekomendasi sistem: G

In [811]:
#AKURASI
print(f"Akurasi : {((i-1)/(stop-1)) * 100}%")

Akurasi : 83.0%


In [812]:
i = 1
for index in hasil_tabel.index:
    for column in hasil_tabel.columns:
        if column != "Riil":
            if hasil_tabel.loc[index, column] != hasil_tabel.loc[index, "Riil"]:
                pil1 = data_test.loc[index, 'Pilihan I']
                pil2 = data_test.loc[index, 'Pilihan II']
                pil3 = data_test.loc[index, 'Pilihan III']
                terima = hasil_tabel.loc[index, "Riil"]
                print(f"""{i}
                index: {index}
                rekomendasi sistem: {hasil_tabel.loc[index, column]}
                pilihan I: {pil1}
                pilihan II: {pil2}
                pilihan III: {pil3}
                diterima dijurusan: {terima}
                """)
                print()
                i += 1

1
                index: 138
                rekomendasi sistem: TEKNIK MESIN - UI (IPA)
                pilihan I: TEKNIK PERKAPALAN - UI (IPA)
                pilihan II: TEKNIK MESIN - UI (IPA)
                pilihan III: TEKNIK INDUSTRI - UNDIP (IPA)
                diterima dijurusan: TEKNIK PERKAPALAN - UI (IPA)
                

2
                index: 344
                rekomendasi sistem: HUKUM - UNAIR (IPS)
                pilihan I: PSIKOLOGI - UNAIR (IPS)
                pilihan II: HUKUM - UNAIR (IPS)
                pilihan III: KOSONG
                diterima dijurusan: PSIKOLOGI - UNAIR (IPS)
                

3
                index: 366
                rekomendasi sistem: STATISTIKA - UNAIR (IPA)
                pilihan I: AKUNTANSI - UNAIR (IPS)
                pilihan II: STATISTIKA - UNAIR (IPA)
                pilihan III: MATEMATIKA - UM MALANG (IPA)
                diterima dijurusan: AKUNTANSI - UNAIR (IPS)
                

4
                index: 367
    